### Add Offset Metadata fields

This notebook is designed for use in ArcGIS Pro 2.9+.  See [this topic](https://pro.arcgis.com/en/pro-app/2.8/arcpy/get-started/pro-notebooks.htm) For information on how to use Jupyter notebooks in ArcGIS Pro.

- Prior to running this script you should ensure that the [Add GPS Metadata Fields](https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/add-gps-metadata-fields.htm) Geoprocessing tool has been run on the `input_feature_class`. 
- Use this script to update an existing point feature class to support offset workflows
- This adds a series of fields and necessary coded value domains

### Steps
1. Update the `input_feaature_class` variable
2. Run all cells

In [70]:
# update to location of line or polygon feature class
input_feature_class = r"C:\Users\doug-m\Documents\ArcGIS\Projects\MyProject\MyGDB.gdb\water_valves"

In [71]:
import arcpy
import os

def get_geodatabase_path(input_feature_class):
    """
    Gets the parent geodatabase of the layer
    :param input_feature_class: (string) The feature layer to get the parent database of
    :return: (string) The path to the geodatabase
    """
    workspace = os.path.dirname(arcpy.Describe(input_feature_class).catalogPath)
    if [any(ext) for ext in ('.gdb', '.mdb', '.sde') if ext in os.path.splitext(workspace)]:
        return workspace
    else:
        return os.path.dirname(workspace)

In [72]:
def check_and_create_domains(geodatabase):
    """
    Checks if the Fix Type domain already exists.
    If the domains do not exist, they are created
    :param geodatabase: (string) the path to the geodatabase to check
    :return:
    """
    domains = arcpy.da.ListDomains(geodatabase)
    domain_names = [domain.name.lower() for domain in domains]
    if 'esri_fix_type_domain' in domain_names:
        for domain in domains:
            if domain.name.lower() == 'esri_fix_type_domain':
                # check if cvs 0,1,2,4,5 are in the codedValues
                values = [cv for cv in domain.codedValues]
                if not {0, 1, 2, 4, 5}.issubset(values):
                    return "esri_fix_type_domain is missing a coded value pair."
    else:
        # Add the domain and values
      
        arcpy.CreateDomain_management(in_workspace=geodatabase,
                                      domain_name="esri_fix_type_domain",
                                      domain_description="Fix Type",
                                      field_type="SHORT",
                                      domain_type="CODED",
                                      split_policy="DEFAULT",
                                      merge_policy="DEFAULT")

        arcpy.AddCodedValueToDomain_management(in_workspace=geodatabase,
                                               domain_name="esri_fix_type_domain",
                                               code="0",
                                               code_description="Fix not valid")
        arcpy.AddCodedValueToDomain_management(in_workspace=geodatabase,
                                               domain_name="esri_fix_type_domain",
                                               code="1",
                                               code_description="GPS")
        arcpy.AddCodedValueToDomain_management(in_workspace=geodatabase,
                                               domain_name="esri_fix_type_domain",
                                               code="2",
                                               code_description="Differential GPS")
        arcpy.AddCodedValueToDomain_management(in_workspace=geodatabase,
                                               domain_name="esri_fix_type_domain",
                                               code="4",
                                               code_description="RTK Fixed")
        arcpy.AddCodedValueToDomain_management(in_workspace=geodatabase,
                                               domain_name="esri_fix_type_domain",
                                               code="5",
                                               code_description="RTK Float")
        
    if 'esri_offset_method_domain' in domain_names:
        for domain in domains:
            if domain.name.lower() == 'esri_offset_method_domain':
                # check if offset methods are in the codedValues
                values = [cv for cv in domain.codedValues]
                if not {'azimuth-azimuth', 
                        'distance-azimuth', 
                        'distance-distance', 
                        'distance-angle', 
                        'azimuth-azimuth-backsight', 
                        'distance-azimuth-backsight',
                        'distance-distance-backsight',
                        'user-defined'}.issubset(values):
                    return "esri_offset_method_domain is missing a coded value pair."
    else:
        # Add the domain and values
      
        arcpy.CreateDomain_management(in_workspace=geodatabase,
                                      domain_name="esri_offset_method_domain",
                                      domain_description="Offset Method Domain",
                                      field_type="TEXT",
                                      domain_type="CODED",
                                      split_policy="DEFAULT",
                                      merge_policy="DEFAULT")

        arcpy.AddCodedValueToDomain_management(in_workspace=geodatabase,
                                               domain_name="esri_offset_method_domain",
                                               code="azimuth-azimuth",
                                               code_description="Azimuth-Azimuth")
        arcpy.AddCodedValueToDomain_management(in_workspace=geodatabase,
                                               domain_name="esri_offset_method_domain",
                                               code="distance-angle",
                                               code_description="Distance-Angle")
        arcpy.AddCodedValueToDomain_management(in_workspace=geodatabase,
                                               domain_name="esri_offset_method_domain",
                                               code="distance-azimuth",
                                               code_description="Distance-Azimuth")
        arcpy.AddCodedValueToDomain_management(in_workspace=geodatabase,
                                               domain_name="esri_offset_method_domain",
                                               code="distance-distance",
                                               code_description="Distance-Distance")
        arcpy.AddCodedValueToDomain_management(in_workspace=geodatabase,
                                               domain_name="esri_offset_method_domain",
                                               code="azimuth-azimuth-backsight",
                                               code_description="Azimuth-Azimuth-Backsight")
        arcpy.AddCodedValueToDomain_management(in_workspace=geodatabase,
                                               domain_name="esri_offset_method_domain",
                                               code="distance-azimuth-backsight",
                                               code_description="Distance-Azimuth-Backsight")
        arcpy.AddCodedValueToDomain_management(in_workspace=geodatabase,
                                               domain_name="esri_offset_method_domain",
                                               code="distance-distance-backsight",
                                               code_description="Distance-Distance-Backsight")
        arcpy.AddCodedValueToDomain_management(in_workspace=geodatabase,
                                               domain_name="esri_offset_method_domain",
                                               code="user-defined",
                                               code_description="User-defined")        

In [ ]:
def add_offset_fields(feature_class):
    """
    This adds specific fields required for laser offset workflows
        This will report errors if:
            1) Any of the fields already exist
            2) The input layer is not a point layer
            3) The layer is not found
            4) The layer is a shapefile
    """

    if not arcpy.Exists(feature_class):
        return "Feature layer: {} not found!".format(feature_class)
    if arcpy.Describe(feature_class).shapeType != "Point":
        return "Feature layer: {} is not a point layer.".format(feature_class)
    if arcpy.Describe(feature_class).dataType == "ShapeFile":
        return "Shapefiles are not supported."

    # Check the domains to see if they exist and are valid
    # will update if necessary
    geodatabase = get_geodatabase_path(feature_class)
    check_and_create_domains(geodatabase)    

    # Add GNSS metadata fields
    existing_fields = [field.name.lower() for field in arcpy.ListFields(feature_class)]

    if 'esrisnsr_offset_method' not in existing_fields:
        arcpy.AddField_management(feature_class,
                                  'esrisnsr_offset_method',
                                  field_type="TEXT",
                                  field_alias='Offset method',
                                  field_is_nullable="NULLABLE",
                                  field_domain="esri_offset_method_domain"
                                  )

    if 'esrisnsr_offset_device' not in existing_fields:
        arcpy.AddField_management(feature_class,
                                  'esrisnsr_offset_device',
                                  field_type="TEXT",
                                  field_alias='Rangefinder name',
                                  field_is_nullable="NULLABLE"
                                  )

    if 'esrignss_antenna_height' not in existing_fields:
        arcpy.AddField_management(feature_class,
                                  'esrignss_antenna_height',
                                  field_type="DOUBLE",
                                  field_alias='GNSS Antenna height (m)',
                                  field_is_nullable="NULLABLE"
                                  )

    if 'esrisnsr_offset_laser_height' not in existing_fields:
        arcpy.AddField_management(feature_class,
                                  'esrisnsr_offset_laser_height',
                                  field_type="DOUBLE",
                                  field_alias='Rangefinder height (m)',
                                  field_is_nullable="NULLABLE"
                                  )

    if 'esrisnsr_offset_laser_target_ht' not in existing_fields:
        arcpy.AddField_management(feature_class,
                                  'esrisnsr_offset_laser_target_ht',
                                  field_type="DOUBLE",
                                  field_alias='Laser target height (m)',
                                  field_is_nullable="NULLABLE"
                                  )

    if 'esrisnsr_offset_mag_dec' not in existing_fields:
        arcpy.AddField_management(feature_class,
                                  'esrisnsr_offset_mag_dec',
                                  field_type="DOUBLE",
                                  field_alias='Magnetic declination (°)',
                                  field_is_nullable="NULLABLE"
                                  )
        
    if 'esrignss_cp1_latitude' not in existing_fields:
        arcpy.AddField_management(feature_class,
                                  'esrignss_cp1_latitude',
                                  field_type="DOUBLE",
                                  field_alias='CP1 Latitude',
                                  field_is_nullable="NULLABLE"
                                  )

    if 'esrignss_cp1_longitude' not in existing_fields:
        arcpy.AddField_management(feature_class,
                                  'esrignss_cp1_longitude',
                                  field_type="DOUBLE",
                                  field_alias='CP1 Longitude',
                                  field_is_nullable="NULLABLE"
                                  )       

    if 'esrignss_cp1_altitude' not in existing_fields:
        arcpy.AddField_management(feature_class,
                                  'esrignss_cp1_altitude',
                                  field_type="DOUBLE",
                                  field_alias='CP1 Ellipsoidal height (m)',
                                  field_is_nullable="NULLABLE"
                                  )

    if 'esrignss_cp1_h_rms' not in existing_fields:
        arcpy.AddField_management(feature_class,
                                  'esrignss_cp1_h_rms',
                                  field_type="DOUBLE",
                                  field_alias='CP1 Horizontal accuracy (m)',
                                  field_is_nullable="NULLABLE"
                                  )     

    if 'esrignss_cp1_v_rms' not in existing_fields:
        arcpy.AddField_management(feature_class,
                                  'esrignss_cp1_v_rms',
                                  field_type="DOUBLE",
                                  field_alias='CP1 Vertical accuracy (m)',
                                  field_is_nullable="NULLABLE"
                                  )

    if 'esrignss_cp1_fixdatetime' not in existing_fields:
        arcpy.AddField_management(feature_class,
                                  'esrignss_cp1_fixdatetime',
                                  field_type="DATE",
                                  field_alias='CP1 Fix time',
                                  field_is_nullable="NULLABLE"
                                  )     

    if 'esrignss_cp1_fixtype' not in existing_fields:
        arcpy.AddField_management(feature_class,
                                  'esrignss_cp1_fixtype',
                                  field_type="SHORT",
                                  field_alias='CP1 Fix type',
                                  field_is_nullable="NULLABLE",
                                  field_domain="esri_fix_type_domain"
                                  )     

    if 'esrignss_cp1_corr_age' not in existing_fields:
        arcpy.AddField_management(feature_class,
                                  'esrignss_cp1_corr_age',
                                  field_type="DOUBLE",
                                  field_alias='CP1 Correction age',
                                  field_is_nullable="NULLABLE"
                                  )

    if 'esrignss_cp1_stationid' not in existing_fields:
        arcpy.AddField_management(feature_class,
                                  'esrignss_cp1_stationid',
                                  field_type="SHORT",
                                  field_alias='CP1 Station ID',
                                  field_is_nullable="NULLABLE"
                                  )    

    if 'esrignss_cp1_numsats' not in existing_fields:
        arcpy.AddField_management(feature_class,
                                  'esrignss_cp1_numsats',
                                  field_type="DOUBLE",
                                  field_alias='CP1 Number of satellites',
                                  field_is_nullable="NULLABLE"
                                  )

    if 'esrignss_cp1_pdop' not in existing_fields:
        arcpy.AddField_management(feature_class,
                                  'esrignss_cp1_pdop',
                                  field_type="DOUBLE",
                                  field_alias='CP1 PDOP',
                                  field_is_nullable="NULLABLE"
                                  )    

    if 'esrignss_cp1_hdop' not in existing_fields:
        arcpy.AddField_management(feature_class,
                                  'esrignss_cp1_hdop',
                                  field_type="DOUBLE",
                                  field_alias='CP1 HDOP',
                                  field_is_nullable="NULLABLE"
                                  )    
        
    if 'esrignss_cp1_vdop' not in existing_fields:
        arcpy.AddField_management(feature_class,
                                  'esrignss_cp1_vdop',
                                  field_type="DOUBLE",
                                  field_alias='CP1 VDOP',
                                  field_is_nullable="NULLABLE"
                                  )        

    if 'esrignss_cp1_avg_h_rms' not in existing_fields:
        arcpy.AddField_management(feature_class,
                                  'esrignss_cp1_avg_h_rms',
                                  field_type="DOUBLE",
                                  field_alias='CP1 Average horizontal accuracy (m)',
                                  field_is_nullable="NULLABLE"
                                  )    

    if 'esrignss_cp1_avg_v_rms' not in existing_fields:
        arcpy.AddField_management(feature_class,
                                  'esrignss_cp1_avg_v_rms',
                                  field_type="DOUBLE",
                                  field_alias='CP1 Average vertical accuracy (m)',
                                  field_is_nullable="NULLABLE"
                                  )    

    if 'esrignss_cp1_avg_positions' not in existing_fields:
        arcpy.AddField_management(feature_class,
                                  'esrignss_cp1_avg_positions',
                                  field_type="SHORT",
                                  field_alias='CP1 Averaged positions',
                                  field_is_nullable="NULLABLE"
                                  )

    if 'esrignss_cp1_h_stddev' not in existing_fields:
        arcpy.AddField_management(feature_class,
                                  'esrignss_cp1_h_stddev',
                                  field_type="DOUBLE",
                                  field_alias='CP1 Standard deviation (m)',
                                  field_is_nullable="NULLABLE"
                                  )    

    if 'esrisnsr_cp1_slope_dist' not in existing_fields:
        arcpy.AddField_management(feature_class,
                                  'esrisnsr_cp1_slope_dist',
                                  field_type="DOUBLE",
                                  field_alias='CP1 Laser slope distance (m)',
                                  field_is_nullable="NULLABLE"
                                  )    
        
    if 'esrisnsr_cp1_inclination' not in existing_fields:
        arcpy.AddField_management(feature_class,
                                  'esrisnsr_cp1_inclination',
                                  field_type="DOUBLE",
                                  field_alias='CP1 Laser inclination (°)',
                                  field_is_nullable="NULLABLE"
                                  )    
        
    if 'esrisnsr_cp1_azimuth' not in existing_fields:
        arcpy.AddField_management(feature_class,
                                  'esrisnsr_cp1_azimuth',
                                  field_type="DOUBLE",
                                  field_alias='CP1 Laser azimuth (°)',
                                  field_is_nullable="NULLABLE"
                                  )        

    if 'esrignss_cp2_latitude' not in existing_fields:
        arcpy.AddField_management(feature_class,
                                  'esrignss_cp2_latitude',
                                  field_type="DOUBLE",
                                  field_alias='CP2 Latitude',
                                  field_is_nullable="NULLABLE"
                                  )

    if 'esrignss_cp2_longitude' not in existing_fields:
        arcpy.AddField_management(feature_class,
                                  'esrignss_cp2_longitude',
                                  field_type="DOUBLE",
                                  field_alias='CP2 Longitude',
                                  field_is_nullable="NULLABLE"
                                  )       

    if 'esrignss_cp2_altitude' not in existing_fields:
        arcpy.AddField_management(feature_class,
                                  'esrignss_cp2_altitude',
                                  field_type="DOUBLE",
                                  field_alias='CP2 Ellipsoidal height (m)',
                                  field_is_nullable="NULLABLE"
                                  )

    if 'esrignss_cp2_h_rms' not in existing_fields:
        arcpy.AddField_management(feature_class,
                                  'esrignss_cp2_h_rms',
                                  field_type="DOUBLE",
                                  field_alias='CP2 Horizontal accuracy (m)',
                                  field_is_nullable="NULLABLE"
                                  )     

    if 'esrignss_cp2_v_rms' not in existing_fields:
        arcpy.AddField_management(feature_class,
                                  'esrignss_cp2_v_rms',
                                  field_type="DOUBLE",
                                  field_alias='CP2 Vertical accuracy (m)',
                                  field_is_nullable="NULLABLE"
                                  )

    if 'esrignss_cp2_fixdatetime' not in existing_fields:
        arcpy.AddField_management(feature_class,
                                  'esrignss_cp2_fixdatetime',
                                  field_type="DATE",
                                  field_alias='CP2 Fix time',
                                  field_is_nullable="NULLABLE"
                                  )     

    if 'esrignss_cp2_fixtype' not in existing_fields:
        arcpy.AddField_management(feature_class,
                                  'esrignss_cp2_fixtype',
                                  field_type="SHORT",
                                  field_alias='CP2 Fix type',
                                  field_is_nullable="NULLABLE",
                                  field_domain="esri_fix_type_domain"
                                  )     

    if 'esrignss_cp2_corr_age' not in existing_fields:
        arcpy.AddField_management(feature_class,
                                  'esrignss_cp2_corr_age',
                                  field_type="DOUBLE",
                                  field_alias='CP2 Correction age',
                                  field_is_nullable="NULLABLE"
                                  )

    if 'esrignss_cp2_stationid' not in existing_fields:
        arcpy.AddField_management(feature_class,
                                  'esrignss_cp2_stationid',
                                  field_type="SHORT",
                                  field_alias='CP2 Station ID',
                                  field_is_nullable="NULLABLE"
                                  )    

    if 'esrignss_cp2_numsats' not in existing_fields:
        arcpy.AddField_management(feature_class,
                                  'esrignss_cp2_numsats',
                                  field_type="DOUBLE",
                                  field_alias='CP2 Number of satellites',
                                  field_is_nullable="NULLABLE"
                                  )

    if 'esrignss_cp2_pdop' not in existing_fields:
        arcpy.AddField_management(feature_class,
                                  'esrignss_cp2_pdop',
                                  field_type="DOUBLE",
                                  field_alias='CP2 PDOP',
                                  field_is_nullable="NULLABLE"
                                  )    

    if 'esrignss_cp2_hdop' not in existing_fields:
        arcpy.AddField_management(feature_class,
                                  'esrignss_cp2_hdop',
                                  field_type="DOUBLE",
                                  field_alias='CP2 HDOP',
                                  field_is_nullable="NULLABLE"
                                  )    
        
    if 'esrignss_cp2_vdop' not in existing_fields:
        arcpy.AddField_management(feature_class,
                                  'esrignss_cp2_vdop',
                                  field_type="DOUBLE",
                                  field_alias='CP2 VDOP',
                                  field_is_nullable="NULLABLE"
                                  )        

    if 'esrignss_cp2_avg_h_rms' not in existing_fields:
        arcpy.AddField_management(feature_class,
                                  'esrignss_cp2_avg_h_rms',
                                  field_type="DOUBLE",
                                  field_alias='CP2 Average horizontal accuracy (m)',
                                  field_is_nullable="NULLABLE"
                                  )    

    if 'esrignss_cp2_avg_v_rms' not in existing_fields:
        arcpy.AddField_management(feature_class,
                                  'esrignss_cp2_avg_v_rms',
                                  field_type="DOUBLE",
                                  field_alias='CP2 Average vertical accuracy (m)',
                                  field_is_nullable="NULLABLE"
                                  )    

    if 'esrignss_cp2_avg_positions' not in existing_fields:
        arcpy.AddField_management(feature_class,
                                  'esrignss_cp2_avg_positions',
                                  field_type="SHORT",
                                  field_alias='CP2 Averaged positions',
                                  field_is_nullable="NULLABLE"
                                  )

    if 'esrignss_cp2_h_stddev' not in existing_fields:
        arcpy.AddField_management(feature_class,
                                  'esrignss_cp2_h_stddev',
                                  field_type="DOUBLE",
                                  field_alias='CP2 Standard deviation (m)',
                                  field_is_nullable="NULLABLE"
                                  )    

    if 'esrisnsr_cp2_slope_dist' not in existing_fields:
        arcpy.AddField_management(feature_class,
                                  'esrisnsr_cp2_slope_dist',
                                  field_type="DOUBLE",
                                  field_alias='CP2 Laser slope distance (m)',
                                  field_is_nullable="NULLABLE"
                                  )    
        
    if 'esrisnsr_cp2_inclination' not in existing_fields:
        arcpy.AddField_management(feature_class,
                                  'esrisnsr_cp2_inclination',
                                  field_type="DOUBLE",
                                  field_alias='CP2 Laser inclination (°)',
                                  field_is_nullable="NULLABLE"
                                  )    
        
    if 'esrisnsr_cp2_azimuth' not in existing_fields:
        arcpy.AddField_management(feature_class,
                                  'esrisnsr_cp2_azimuth',
                                  field_type="DOUBLE",
                                  field_alias='CP2 Laser azimuth (°)',
                                  field_is_nullable="NULLABLE"
                                  )

    # Update GNSS metadata fields with Domains

    domain_fields = [field for field in arcpy.ListFields(feature_class) if field.name == 'esrisnsr_offset_method']

    for field in domain_fields:
        if field.name == 'esrisnsr_offset_method' and not field.domain:
            arcpy.AssignDomainToField_management(feature_class, field, 'esri_offset_method_domain')

    domain_fields = [field for field in arcpy.ListFields(feature_class) if field.name == 'esrignss_cp1_fixtype']

    for field in domain_fields:
        if field.name == 'esrignss_cp1_fixtype' and not field.domain:
            arcpy.AssignDomainToField_management(feature_class, field, 'esri_fix_type_domain')

    domain_fields = [field for field in arcpy.ListFields(feature_class) if field.name == 'esrignss_cp2_fixtype']

    for field in domain_fields:
        if field.name == 'esrignss_cp2_fixtype' and not field.domain:
            arcpy.AssignDomainToField_management(feature_class, field, 'esri_fix_type_domain')

    return "Successfully added offset metadata fields and domains."

In [ ]:
add_offset_fields(input_feature_class)